In [1]:
DEBUG = False

In [2]:
import os
import sys
sys.path = [
    './efficientnet/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path

In [3]:
sys.path

['./efficientnet/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
 '/home/under/ddw02141/PANDA',
 '/home/under/ddw02141/miniconda3/envs/sangwon/lib/python37.zip',
 '/home/under/ddw02141/miniconda3/envs/sangwon/lib/python3.7',
 '/home/under/ddw02141/miniconda3/envs/sangwon/lib/python3.7/lib-dynload',
 '',
 '/home/under/ddw02141/.local/lib/python3.7/site-packages',
 '/home/under/ddw02141/miniconda3/envs/sangwon/lib/python3.7/site-packages',
 '/home/under/ddw02141/.local/lib/python3.7/site-packages/IPython/extensions',
 '/home/under/ddw02141/.ipython']

In [4]:
import time
import skimage.io
import numpy as np
import pandas as pd
import cv2
import PIL.Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler, RandomSampler, SequentialSampler
from warmup_scheduler import GradualWarmupScheduler
from efficientnet_pytorch import model as enet
import albumentations
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
from tqdm import tqdm_notebook as tqdm
import os
import warnings 
warnings.filterwarnings('ignore')

# Config

In [5]:
data_dir = './prostate-cancer-grade-assessment/'
df_train = pd.read_csv(os.path.join(data_dir, 'train.csv'))
image_folder = os.path.join(data_dir, 'train_images/')

kernel_type = 'efficientnet_b1_36x256x256_final'

enet_type = 'efficientnet-b1'
fold = 0
tile_size = 256
image_size = 256
n_tiles = 36
batch_size =8
num_workers = 16
out_dim = 5
init_lr = 3e-4
warmup_factor = 10

warmup_epo = 1
n_epochs = 1 if DEBUG else 50
df_train = df_train.sample(100).reset_index(drop=True) if DEBUG else df_train

device = torch.device('cuda')

# Create Folds

In [6]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "2,3,4,5"

In [7]:
skf = StratifiedKFold(5, shuffle=True, random_state=42)
df_train['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(df_train, df_train['isup_grade'])):
    df_train.loc[valid_idx, 'fold'] = i
df_train.head()

,image_id,data_provider,isup_grade,gleason_score,fold
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0,4
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0,0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4,3
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4,4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0,4


# Model

In [8]:
pretrained_model = {
    'efficientnet-b1': './efficientnet/efficientnet-b1-dbc7070a.pth'
}

In [9]:
class enetv2(nn.Module):
    def __init__(self, backbone, out_dim):
        super(enetv2, self).__init__()
        self.enet = enet.EfficientNet.from_name(backbone)
        self.enet.load_state_dict(torch.load(pretrained_model[backbone]))

        self.myfc = nn.Linear(self.enet._fc.in_features, out_dim)
        self.enet._fc = nn.Identity()

    def extract(self, x):
        return self.enet(x)

    def forward(self, x):
        x = self.extract(x)
        x = self.myfc(x)
        return x

# Dataset

In [10]:
def get_tiles(img, mode=0):
        result = []
        h, w, c = img.shape
        pad_h = (tile_size - h % tile_size) % tile_size + ((tile_size * mode) // 2)
        pad_w = (tile_size - w % tile_size) % tile_size + ((tile_size * mode) // 2)

        img2 = np.pad(img,[[pad_h // 2, pad_h - pad_h // 2], [pad_w // 2,pad_w - pad_w//2], [0,0]], constant_values=255)
        img3 = img2.reshape(
            img2.shape[0] // tile_size,
            tile_size,
            img2.shape[1] // tile_size,
            tile_size,
            3
        )

        img3 = img3.transpose(0,2,1,3,4).reshape(-1, tile_size, tile_size,3)
        n_tiles_with_info = (img3.reshape(img3.shape[0],-1).sum(1) < tile_size ** 2 * 3 * 255).sum()
        if len(img3) < n_tiles:
            img3 = np.pad(img3,[[0,n_tiles-len(img3)],[0,0],[0,0],[0,0]], constant_values=255)
        idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:n_tiles]
        img3 = img3[idxs]
        for i in range(len(img3)):
            result.append({'img':img3[i], 'idx':i})
        return result, n_tiles_with_info >= n_tiles


class PANDADataset(Dataset):
    def __init__(self,
                 df,
                 image_size,
                 n_tiles=n_tiles,
                 tile_mode=0,
                 rand=False,
                 transform=None,
                ):

        self.df = df.reset_index(drop=True)
        self.image_size = image_size
        self.n_tiles = n_tiles
        self.tile_mode = tile_mode
        self.rand = rand
        self.transform = transform

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img_id = row.image_id
        
        tiff_file = os.path.join(image_folder, f'{img_id}.tiff')
        image = skimage.io.MultiImage(tiff_file)[-2]
        tiles, OK = get_tiles(image, self.tile_mode)

        if self.rand:
            idxes = np.random.choice(list(range(self.n_tiles)), self.n_tiles, replace=False)
        else:
            idxes = list(range(self.n_tiles))

        n_row_tiles = int(np.sqrt(self.n_tiles))
        images = np.zeros((image_size * n_row_tiles, image_size * n_row_tiles, 3))
        for h in range(n_row_tiles):
            for w in range(n_row_tiles):
                i = h * n_row_tiles + w
    
                if len(tiles) > idxes[i]:
                    this_img = tiles[idxes[i]]['img']
                else:
                    this_img = np.ones((self.image_size, self.image_size, 3)).astype(np.uint8) * 255
                this_img = 255 - this_img
                if self.transform is not None:
                    this_img = self.transform(image=this_img)['image']
                h1 = h * image_size
                w1 = w * image_size
                images[h1:h1+image_size, w1:w1+image_size] = this_img

        if self.transform is not None:
            images = self.transform(image=images)['image']
        images = images.astype(np.float32)
        images /= 255
        images = images.transpose(2, 0, 1)

        label = np.zeros(5).astype(np.float32)
        label[:row.isup_grade] = 1.
        return torch.tensor(images), torch.tensor(label)


# Augmentations

In [11]:
transforms_train = albumentations.Compose([
    albumentations.Transpose(p=0.5),
    albumentations.VerticalFlip(p=0.5),
    albumentations.HorizontalFlip(p=0.5),
])
transforms_val = albumentations.Compose([])

dataset_show = PANDADataset(df_train, image_size, n_tiles, 0, transform=transforms_train)
from pylab import rcParams
rcParams['figure.figsize'] = 20,10
for i in range(2):
    f, axarr = plt.subplots(1,5)
    for p in range(5):
        idx = np.random.randint(0, len(dataset_show))
        img, label = dataset_show[idx]
        axarr[p].imshow(1. - img.transpose(0, 1).transpose(1,2).squeeze())
        axarr[p].set_title(str(sum(label)))


# Loss

In [12]:
criterion = nn.BCEWithLogitsLoss()

# Train & Val

In [13]:
def train_epoch(loader, optimizer):

    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        
        data, target = data.to(device), target.to(device)
        loss_func = criterion
        optimizer.zero_grad()
        logits = model(data)
        loss = loss_func(logits, target)
        loss.backward()
        optimizer.step()

        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))
    return train_loss


def val_epoch(loader, get_output=False):

    model.eval()
    val_loss = []
    LOGITS = []
    PREDS = []
    TARGETS = []

    with torch.no_grad():
        for (data, target) in tqdm(loader):
            data, target = data.to(device), target.to(device)
            logits = model(data)

            loss = criterion(logits, target)

            pred = logits.sigmoid().sum(1).detach().round()
            LOGITS.append(logits)
            PREDS.append(pred)
            TARGETS.append(target.sum(1))

            val_loss.append(loss.detach().cpu().numpy())
        val_loss = np.mean(val_loss)

    LOGITS = torch.cat(LOGITS).cpu().numpy()
    PREDS = torch.cat(PREDS).cpu().numpy()
    TARGETS = torch.cat(TARGETS).cpu().numpy()
    acc = (PREDS == TARGETS).mean() * 100.
    
    qwk = cohen_kappa_score(PREDS, TARGETS, weights='quadratic')
    qwk_k = cohen_kappa_score(PREDS[df_valid['data_provider'] == 'karolinska'], df_valid[df_valid['data_provider'] == 'karolinska'].isup_grade.values, weights='quadratic')
    qwk_r = cohen_kappa_score(PREDS[df_valid['data_provider'] == 'radboud'], df_valid[df_valid['data_provider'] == 'radboud'].isup_grade.values, weights='quadratic')
    print('qwk', qwk, 'qwk_k', qwk_k, 'qwk_r', qwk_r)

    if get_output:
        return LOGITS
    else:
        return val_loss, acc, qwk

    

# Create Dataloader & Model & Optimizer

In [14]:
train_idx = np.where((df_train['fold'] != fold))[0]
valid_idx = np.where((df_train['fold'] == fold))[0]

df_this  = df_train.loc[train_idx]
df_valid = df_train.loc[valid_idx]

dataset_train = PANDADataset(df_this , image_size, n_tiles, transform=transforms_train,  rand=True)
dataset_valid = PANDADataset(df_valid, image_size, n_tiles, transform=transforms_val,  rand=True)

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, sampler=RandomSampler(dataset_train), num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=batch_size, sampler=SequentialSampler(dataset_valid), num_workers=num_workers)

model = enetv2(enet_type, out_dim=out_dim)
model = nn.DataParallel(model, device_ids = [0,1,2,3])
model = model.to(device)


optimizer = optim.Adam(model.parameters(), lr=init_lr/warmup_factor)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs-warmup_epo)
scheduler = GradualWarmupScheduler(optimizer, multiplier=warmup_factor, total_epoch=warmup_epo, after_scheduler=scheduler_cosine)

print(len(dataset_train), len(dataset_valid))

8492 2124


In [15]:
if 'checkpoints' not in os.listdir():
    !mkdir checkpoints

if 'logs' not in os.listdir():
    !mkdir logs

# Run Training

In [16]:
qwk_max = 0.
best_file = f'checkpoints/{kernel_type}_best_fold{fold}.pth'
for epoch in range(1, n_epochs+1):
    print(time.ctime(), 'Epoch:', epoch)
    scheduler.step(epoch-1)

    train_loss = train_epoch(train_loader, optimizer)
    val_loss, acc, qwk = val_epoch(valid_loader)

    content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {np.mean(train_loss):.5f}, val loss: {np.mean(val_loss):.5f}, acc: {(acc):.5f}, qwk: {(qwk):.5f}'
    print(content)
    with open(f'logs/log_{kernel_type}.txt', 'a') as appender:
        appender.write(content + '\n')

    if qwk > qwk_max:
        print('score2 ({:.6f} --> {:.6f}).  Saving model ...'.format(qwk_max, qwk))
        torch.save(model.state_dict(), best_file)
        qwk_max = qwk

torch.save(model.state_dict(), os.path.join(f'{kernel_type}_final_fold{fold}.pth'))

Fri Jul 31 01:12:31 2020 Epoch: 1



qwk 0.6852177459427153 qwk_k 0.6375299046293832 qwk_r 0.6319657437028228
Fri Jul 31 01:22:48 2020 Epoch 1, lr: 0.0000300, train loss: 0.48728, val loss: 0.35194, acc: 32.67420, qwk: 0.68522
score2 (0.000000 --> 0.685218).  Saving model ...
Fri Jul 31 01:22:48 2020 Epoch: 2



qwk 0.7454934308877695 qwk_k 0.7113386956404304 qwk_r 0.6863973646286833
Fri Jul 31 01:32:59 2020 Epoch 2, lr: 0.0003000, train loss: 0.36829, val loss: 0.31629, acc: 36.62900, qwk: 0.74549
score2 (0.685218 --> 0.745493).  Saving model ...
Fri Jul 31 01:33:00 2020 Epoch: 3



qwk 0.8383836477378939 qwk_k 0.8367210841928616 qwk_r 0.7976045761153205
Fri Jul 31 01:43:09 2020 Epoch 3, lr: 0.0000300, train loss: 0.27242, val loss: 0.24789, acc: 56.82674, qwk: 0.83838
score2 (0.745493 --> 0.838384).  Saving model ...
Fri Jul 31 01:43:09 2020 Epoch: 4



qwk 0.8017132308410737 qwk_k 0.7986607132275255 qwk_r 0.7575776722247339
Fri Jul 31 01:53:26 2020 Epoch 4, lr: 0.0002988, train loss: 0.28789, val loss: 0.28162, acc: 61.95857, qwk: 0.80171
Fri Jul 31 01:53:26 2020 Epoch: 5


...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




qwk 0.8757704353362852 qwk_k 0.8909242406207053 qwk_r 0.8349276150631681
Fri Jul 31 09:13:44 2020 Epoch 47, lr: 0.0000049, train loss: 0.04592, val loss: 0.33006, acc: 69.49153, qwk: 0.87577
Fri Jul 31 09:13:44 2020 Epoch: 48


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




qwk 0.8742718433026364 qwk_k 0.8901641623560541 qwk_r 0.8318805983722093
Fri Jul 31 09:34:14 2020 Epoch 49, lr: 0.0000012, train loss: 0.04644, val loss: 0.32033, acc: 69.77401, qwk: 0.87427
Fri Jul 31 09:34:14 2020 Epoch: 50


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Clean Train log

Fri Jul 31 01:22:48 2020 Epoch 1, lr: 0.0000300, train loss: 0.48728, val loss: 0.35194, acc: 32.67420, qwk: 0.68522

Fri Jul 31 01:32:59 2020 Epoch 2, lr: 0.0003000, train loss: 0.36829, val loss: 0.31629, acc: 36.62900, qwk: 0.74549

Fri Jul 31 01:43:09 2020 Epoch 3, lr: 0.0000300, train loss: 0.27242, val loss: 0.24789, acc: 56.82674, qwk: 0.83838

Fri Jul 31 01:53:26 2020 Epoch 4, lr: 0.0002988, train loss: 0.28789, val loss: 0.28162, acc: 61.95857, qwk: 0.80171

Fri Jul 31 02:03:43 2020 Epoch 5, lr: 0.0002972, train loss: 0.26203, val loss: 0.25778, acc: 57.25047, qwk: 0.83672

Fri Jul 31 02:13:58 2020 Epoch 6, lr: 0.0002951, train loss: 0.24874, val loss: 0.28150, acc: 48.44633, qwk: 0.80569

Fri Jul 31 02:24:14 2020 Epoch 7, lr: 0.0002924, train loss: 0.23590, val loss: 0.24244, acc: 59.51036, qwk: 0.84836

Fri Jul 31 02:34:28 2020 Epoch 8, lr: 0.0002890, train loss: 0.22646, val loss: 0.24412, acc: 61.39360, qwk: 0.84483

Fri Jul 31 02:44:41 2020 Epoch 9, lr: 0.0002851, train loss: 0.21952, val loss: 0.23171, acc: 62.00565, qwk: 0.85636

Fri Jul 31 02:54:56 2020 Epoch 10, lr: 0.0002807, train loss: 0.21155, val loss: 0.25910, acc: 61.58192, qwk: 0.84736

Fri Jul 31 03:05:09 2020 Epoch 11, lr: 0.0002757, train loss: 0.20425, val loss: 0.24255, acc: 62.09981, qwk: 0.85714

Fri Jul 31 03:15:23 2020 Epoch 12, lr: 0.0002702, train loss: 0.19756, val loss: 0.22886, acc: 61.39360, qwk: 0.86287

Fri Jul 31 03:25:38 2020 Epoch 13, lr: 0.0002642, train loss: 0.19369, val loss: 0.24613, acc: 64.87759, qwk: 0.86603

Fri Jul 31 03:35:51 2020 Epoch 14, lr: 0.0002578, train loss: 0.18799, val loss: 0.24163, acc: 66.05461, qwk: 0.84961

Fri Jul 31 03:46:04 2020 Epoch 15, lr: 0.0002508, train loss: 0.17917, val loss: 0.26223, acc: 64.31262, qwk: 0.84652

Fri Jul 31 03:56:17 2020 Epoch 16, lr: 0.0002435, train loss: 0.17006, val loss: 0.23943, acc: 63.84181, qwk: 0.84214

Fri Jul 31 04:06:30 2020 Epoch 17, lr: 0.0002358, train loss: 0.16587, val loss: 0.23266, acc: 64.97175, qwk: 0.86491

Fri Jul 31 04:16:43 2020 Epoch 18, lr: 0.0002278, train loss: 0.16135, val loss: 0.21974, acc: 66.61959, qwk: 0.87617

Fri Jul 31 04:26:57 2020 Epoch 19, lr: 0.0002194, train loss: 0.15672, val loss: 0.23681, acc: 64.35970, qwk: 0.85780

Fri Jul 31 04:37:10 2020 Epoch 20, lr: 0.0002107, train loss: 0.14988, val loss: 0.25504, acc: 62.28814, qwk: 0.85248

Fri Jul 31 04:47:25 2020 Epoch 21, lr: 0.0002018, train loss: 0.14172, val loss: 0.24680, acc: 64.83051, qwk: 0.86191

Fri Jul 31 04:57:37 2020 Epoch 22, lr: 0.0001927, train loss: 0.13453, val loss: 0.31979, acc: 65.16008, qwk: 0.82527

Fri Jul 31 05:07:50 2020 Epoch 23, lr: 0.0001834, train loss: 0.13046, val loss: 0.24314, acc: 65.67797, qwk: 0.86635

Fri Jul 31 05:18:03 2020 Epoch 24, lr: 0.0001739, train loss: 0.12409, val loss: 0.24002, acc: 67.18456, qwk: 0.87073

Fri Jul 31 05:28:18 2020 Epoch 25, lr: 0.0001644, train loss: 0.12092, val loss: 0.27853, acc: 66.00753, qwk: 0.86430

Fri Jul 31 05:38:32 2020 Epoch 26, lr: 0.0001548, train loss: 0.11514, val loss: 0.24585, acc: 67.84369, qwk: 0.87188

Fri Jul 31 05:48:40 2020 Epoch 27, lr: 0.0001452, train loss: 0.10879, val loss: 0.27445, acc: 64.87759, qwk: 0.86390

Fri Jul 31 05:58:52 2020 Epoch 28, lr: 0.0001356, train loss: 0.10443, val loss: 0.26355, acc: 67.56121, qwk: 0.87602

Fri Jul 31 06:09:04 2020 Epoch 29, lr: 0.0001261, train loss: 0.09894, val loss: 0.26092, acc: 67.27872, qwk: 0.87175

Fri Jul 31 06:19:17 2020 Epoch 30, lr: 0.0001166, train loss: 0.09263, val loss: 0.26277, acc: 68.26742, qwk: 0.87605


Fri Jul 31 06:29:35 2020 Epoch 31, lr: 0.0001073, train loss: 0.09059, val loss: 0.27439, acc: 68.17326, qwk: 0.87394

Fri Jul 31 06:39:51 2020 Epoch 32, lr: 0.0000982, train loss: 0.08415, val loss: 0.25896, acc: 67.23164, qwk: 0.87771

Fri Jul 31 06:50:07 2020 Epoch 33, lr: 0.0000893, train loss: 0.07885, val loss: 0.29701, acc: 68.40866, qwk: 0.86528

Fri Jul 31 07:00:20 2020 Epoch 34, lr: 0.0000806, train loss: 0.07550, val loss: 0.29093, acc: 68.22034, qwk: 0.86605

Fri Jul 31 07:10:37 2020 Epoch 35, lr: 0.0000722, train loss: 0.07180, val loss: 0.29977, acc: 68.40866, qwk: 0.86560

Fri Jul 31 07:20:50 2020 Epoch 36, lr: 0.0000642, train loss: 0.06816, val loss: 0.29129, acc: 68.50282, qwk: 0.86896

Fri Jul 31 07:31:06 2020 Epoch 37, lr: 0.0000565, train loss: 0.06662, val loss: 0.30470, acc: 67.89077, qwk: 0.87001

Fri Jul 31 07:41:21 2020 Epoch 38, lr: 0.0000492, train loss: 0.06030, val loss: 0.31497, acc: 67.32580, qwk: 0.86569

Fri Jul 31 07:51:37 2020 Epoch 39, lr: 0.0000422, train loss: 0.06160, val loss: 0.29281, acc: 68.07910, qwk: 0.87695

Fri Jul 31 08:01:52 2020 Epoch 40, lr: 0.0000358, train loss: 0.05638, val loss: 0.31547, acc: 69.11488, qwk: 0.87247

Fri Jul 31 08:12:11 2020 Epoch 41, lr: 0.0000298, train loss: 0.05708, val loss: 0.31981, acc: 69.58569, qwk: 0.87215

Fri Jul 31 08:22:26 2020 Epoch 42, lr: 0.0000243, train loss: 0.05326, val loss: 0.31598, acc: 68.64407, qwk: 0.87265

Fri Jul 31 08:32:43 2020 Epoch 43, lr: 0.0000193, train loss: 0.04858, val loss: 0.32906, acc: 69.06780, qwk: 0.87208

Fri Jul 31 08:42:59 2020 Epoch 44, lr: 0.0000149, train loss: 0.04714, val loss: 0.32263, acc: 69.35028, qwk: 0.87109

Fri Jul 31 08:53:14 2020 Epoch 45, lr: 0.0000110, train loss: 0.04844, val loss: 0.31060, acc: 69.16196, qwk: 0.87609

Fri Jul 31 09:03:30 2020 Epoch 46, lr: 0.0000076, train loss: 0.04556, val loss: 0.32499, acc: 69.91525, qwk: 0.87472

Fri Jul 31 09:13:44 2020 Epoch 47, lr: 0.0000049, train loss: 0.04592, val loss: 0.33006, acc: 69.49153, qwk: 0.87577

Fri Jul 31 09:23:59 2020 Epoch 48, lr: 0.0000028, train loss: 0.04433, val loss: 0.32192, acc: 69.63277, qwk: 0.87289

Fri Jul 31 09:34:14 2020 Epoch 49, lr: 0.0000012, train loss: 0.04644, val loss: 0.32033, acc: 69.77401, qwk: 0.87427

Fri Jul 31 09:44:26 2020 Epoch 50, lr: 0.0000003, train loss: 0.04275, val loss: 0.33012, acc: 69.25612, qwk: 0.87218